In [195]:
import pandas as pd
import os

In [196]:
mes_referencia="07"
ano_referencia="22"

In [197]:
def suma_mes(mes):
    mes_cadena=str(int(mes)+1)
    if len(mes_cadena)==1:
        mes_cadena="0"+ mes_cadena
    return mes_cadena

In [198]:
tramas=[]
for (root,dirs,files) in os.walk('TRAMAS'):
    for name in files:
        if name[0:1]=="M":
            dia=name[12:14]
            mes=name[15:17]
            ano=name[20:22]
            if (mes_referencia==mes and ano_referencia == ano) or (dia=="01" and suma_mes(mes_referencia)==mes):
                tramas.append(root + "\\"+ os.path.join(name))

In [199]:
# pandas concat csv files
df = pd.concat((pd.read_csv(f, sep=";", encoding="latin-1", header=None, skiprows=1) for f in tramas), ignore_index=False)
df.columns =  ["Tipo", "Estacion", "Fecha", "Hora_inicio", "Mezcla", "Premezcla", "Batch", "Receta", "Kg_corcho", "Minutos_pesada", "Kg_DU", "Tiempo_vaciado", "Tiempo_inyeccion_agua", "Kg_Agua", "Tiempo_mezcla_agua", "Tiempo_inyeccion_cola", "Kg_cola", "Tiempo_mezcla_cola", "Tiempo_vaciado_mezcla", "Molde_inicial_ext1", "Molde_final_ext1", "Receta", "Molde_inicial_ext2", "Molde_final_ext2", "Tapones_buenos_g1", "Tapones_malos_g1", "Tiempo_extraccion_g1", "Tapones_buenos_g2", "Tapones_malos_g2", "Tiempo_extraccion_g2", "Tapones_buenos_descartados_g1", "Tapones_buenos_descartados_g2", "nada_1", "nada_2"]

In [200]:
df.drop(columns=["Tipo", "Hora_inicio", "Premezcla", "Batch", "Receta", "Minutos_pesada", "Tiempo_vaciado", "Tiempo_inyeccion_agua", "Kg_Agua", "Tiempo_mezcla_agua", "Tiempo_inyeccion_cola", "Tiempo_mezcla_cola", "Tiempo_vaciado_mezcla", "Molde_inicial_ext1", "Molde_final_ext1", "Receta", "Molde_inicial_ext2", "Molde_final_ext2", "Tiempo_extraccion_g1", "Tiempo_extraccion_g2","nada_1", "nada_2"], inplace=True)	

In [201]:
df=df.loc[:, ["Fecha", "Estacion", "Mezcla", "Kg_corcho", "Kg_cola", "Kg_DU", "Tapones_buenos_g1", "Tapones_buenos_g2", "Tapones_malos_g1", "Tapones_malos_g2", "Tapones_buenos_descartados_g1", "Tapones_buenos_descartados_g2"]].sort_values(by=["Fecha", "Estacion", "Mezcla"])

In [202]:
df.reset_index(level=0, inplace=True)
df.drop(columns=["index"], inplace=True)

In [203]:
tf=(df["Tapones_malos_g1"]==65492)
error_cantidad_malos= round((len(tf[tf==True])/len(df))*100,2)

In [204]:
df.loc[tf, "Tapones_malos_g1"]=0
df2 = df.copy() 

In [205]:
df2.loc[:, "Kg_corcho"]=df2["Kg_corcho"]/1000
df2.loc[:, "Kg_cola"]=df2["Kg_cola"]/1000
df2.loc[:, "Kg_DU"]=df2["Kg_DU"]/1000
df2.loc[:, "Tapones_buenos"]=(df2["Tapones_buenos_g1"]+df2["Tapones_buenos_g2"])/1000
df2.loc[:, "Tapones_malos"]=(df2["Tapones_malos_g1"]+df2["Tapones_malos_g2"])/1000
df2.loc[:, "Tapones_buenos_descartados"]=(df2["Tapones_buenos_descartados_g1"]+df2["Tapones_buenos_descartados_g2"])/1000
df2.drop(columns=["Tapones_buenos_g1", "Tapones_buenos_g2", "Tapones_malos_g1", "Tapones_malos_g2", "Tapones_buenos_descartados_g1", "Tapones_buenos_descartados_g2"], inplace=True)
df2["Estacion"].replace([10], "MA10", inplace=True)
df2["Estacion"].replace([1], "MA11", inplace=True)
df2["Estacion"].replace([12], "MA12", inplace=True)
df2["Estacion"].replace([13], "MA13", inplace=True)
df2["Estacion"].replace([14], "MA14", inplace=True)
df2["Estacion"].replace([15], "MA15", inplace=True)

In [206]:
df2. drop_duplicates(["Fecha", "Estacion", "Mezcla"], keep="first", inplace=True)
df2

,Fecha,Estacion,Mezcla,Kg_corcho,Kg_cola,Kg_DU,Tapones_buenos,Tapones_malos,Tapones_buenos_descartados
0,1/7/22,MA11,1,70.815,22.690,4.604,12.276,0.044,0.000
2,1/7/22,MA11,2,70.388,22.527,4.576,12.012,0.000,0.042
3,1/7/22,MA11,3,71.092,22.771,4.764,11.836,0.000,0.000
5,1/7/22,MA11,4,70.547,22.608,4.586,12.232,0.000,0.000
6,1/7/22,MA11,5,70.956,22.730,4.542,11.704,0.000,0.000
...,...,...,...,...,...,...,...,...,...
15486,9/7/22,MA15,52,69.008,19.322,4.141,10.736,0.000,0.030
15487,9/7/22,MA15,53,68.664,19.231,4.326,13.992,0.000,0.053
15489,9/7/22,MA15,54,69.023,19.332,4.142,11.748,0.088,0.020
15492,9/7/22,MA15,55,68.783,19.261,4.265,11.616,0.000,0.039


In [207]:
#create csv file with commain float fields
df2.to_csv("tramas.csv", sep=";", encoding="latin-1", index=False,decimal=",")